In [1]:
import requests
import pandas as pd
import yaml
import os
import datetime as dt

from secrets_config import api_key


In [2]:
with open("config.yaml") as stream:
    config = yaml.safe_load(stream)

target_cities = config['extract']['target_cities']
salesResults_url = config['extract']['api_urls']['salesResults']
salesResults_listings_url = config['extract']['api_urls']['salesResults_listings']


In [3]:
# extracting listing summaries for the last sales RESULT (Properties&Locations/ Listings)

from sqlalchemy import null

params = {
       "api_key": api_key
    } 

df_results = None

for city in target_cities:
    response_data = []
    url = salesResults_listings_url.replace('{city}', city)

    response = requests.get(url, params=params)

    if response.json() is not None: 
            response_data.extend(response.json())
            print(f"Status: {response.status_code} for {url}")

    temp_df = pd.json_normalize(data=response.json())

    # Adding metadata
    temp_df['City'] = city
    temp_df['URL'] = url
    temp_df['Execution time'] = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    if 'df_results' in locals():
        df_results = pd.concat([df_results, temp_df], ignore_index=True)
    else:
        df_results = temp_df.copy()



Status: 200 for https://api.domain.com.au/v1/salesResults/Sydney/listings
Status: 200 for https://api.domain.com.au/v1/salesResults/Melbourne/listings
Status: 200 for https://api.domain.com.au/v1/salesResults/Adelaide/listings


In [4]:
df_results = pd.concat([df_results[['City', 'URL', 'Execution time']], df_results.iloc[:,:-3]], axis=1)
df_results.head()

,City,URL,Execution time,agencyId,id,propertyDetailsUrl,agencyName,agencyProfilePageUrl,agent,bathrooms,...,result,state,streetName,streetNumber,streetType,suburb,unitNumber,geoLocation.latitude,geoLocation.longitude,price
0,Sydney,https://api.domain.com.au/v1/salesResults/Sydn...,2022-10-04T21:59:44.253885Z,5571,2018055819,https://www.domain.com.au/6-4-buller-road-arta...,Shead Property,https://www.domain.com.au/real-estate-agencies...,Shead Property,1.0,...,AUWD,Nsw,Buller,4,Rd,Artarmon,6,-33.809655,151.180969,NaN
1,Sydney,https://api.domain.com.au/v1/salesResults/Sydn...,2022-10-04T21:59:44.253885Z,15659,2018044684,https://www.domain.com.au/11-39-albert-pde-ash...,Raine & Horne Ashfield,https://www.domain.com.au/real-estate-agencies...,R&H Ashfield,1.0,...,AUHB,Nsw,Albert,39,Pde,Ashfield,11,-33.882008,151.122707,930000.0
2,Sydney,https://api.domain.com.au/v1/salesResults/Sydn...,2022-10-04T21:59:44.253885Z,15659,2018048135,https://www.domain.com.au/10-99-alt-street-ash...,Raine & Horne Ashfield,https://www.domain.com.au/real-estate-agencies...,R&H Ashfield,1.0,...,AUHB,Nsw,Alt,99,St,Ashfield,10,-33.880508,151.129351,700000.0
3,Sydney,https://api.domain.com.au/v1/salesResults/Sydn...,2022-10-04T21:59:44.253885Z,27518,2018032787,https://www.domain.com.au/49-5-benalla-avenue-...,Belle Property Ashfield,https://www.domain.com.au/real-estate-agencies...,Bell Ashfield,1.0,...,AUSP,Nsw,Benalla,5,Av,Ashfield,49,-33.885341,151.120431,680000.0
4,Sydney,https://api.domain.com.au/v1/salesResults/Sydn...,2022-10-04T21:59:44.253885Z,4763,2018053110,https://www.domain.com.au/94-frederick-street-...,Hudson McHugh,https://www.domain.com.au/real-estate-agencies...,Hudson McHugh,2.0,...,AUSP,Nsw,Frederick,94,St,Ashfield,NaN,-33.880356,151.122730,1740000.0


In [5]:
df_results.columns

Index(['City', 'URL', 'Execution time', 'agencyId', 'id', 'propertyDetailsUrl',
       'agencyName', 'agencyProfilePageUrl', 'agent', 'bathrooms', 'bedrooms',
       'carspaces', 'postcode', 'propertyType', 'result', 'state',
       'streetName', 'streetNumber', 'streetType', 'suburb', 'unitNumber',
       'geoLocation.latitude', 'geoLocation.longitude', 'price'],
      dtype='object')

In [6]:
#Loading into the Postgres

from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float, JSON # https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from secrets_config import db_user, db_password, db_server_name, db_database_name


In [7]:
# create connection to database 
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5432,
    database = db_database_name, 
)
engine = create_engine(connection_url)


In [8]:
df_results.dtypes

City                      object
URL                       object
Execution time            object
agencyId                   int64
id                         int64
propertyDetailsUrl        object
agencyName                object
agencyProfilePageUrl      object
agent                     object
bathrooms                float64
bedrooms                   int64
carspaces                float64
postcode                  object
propertyType              object
result                    object
state                     object
streetName                object
streetNumber              object
streetType                object
suburb                    object
unitNumber                object
geoLocation.latitude     float64
geoLocation.longitude    float64
price                    float64
dtype: object

In [46]:
# specify target table schema 
raw_table = "raw_listings_result"

# create meta object 
meta = MetaData()

# specify trade data table schema 
raw_listings_result = Table(
    raw_table, meta, 
    Column("City", String, primary_key=True),
    Column("Execution time", String),
    Column("URL", String),
    Column("id", Integer, primary_key=True),
    Column("agencyId", Integer),
    Column("propertyDetailsUrl", String),
    Column("agencyName", String),
    Column("agencyProfilePageUrl", String),
    Column("agent", String),
    Column("bathrooms", Float),
    Column("bedrooms", Integer),
    Column("carspaces", Float),
    Column("postcode", String),
    Column("propertyType", String),
    Column("result", String, primary_key=True),
    Column("streetName", String),
    Column("streetNumber", String),
    Column("streetType", String),
    Column("suburb", String),
    Column("geoLocation.latitude", Float),
    Column("geoLocation.longitude", Float),
    Column("unitNumber", String),
    Column("state", String),
    Column("price", Float)
)

# creates table if it does not exist 
meta.create_all(engine)


In [47]:

# To Do: the upsert does not work. Take a look at the df_listings and "id"

insert_statement = postgresql.insert(raw_listings_result).values(df_results.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['id', 'result', 'City'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['id', 'result', 'City']})
with engine.connect() as connection: 
    connection.execute(upsert_statement)

ProgrammingError: (pg8000.dbapi.ProgrammingError) {'S': 'ERROR', 'V': 'ERROR', 'C': '42P10', 'M': 'there is no unique or exclusion constraint matching the ON CONFLICT specification', 'F': 'plancat.c', 'L': '844', 'R': 'infer_arbiter_indexes'}
[SQL: INSERT INTO raw_listings_result ("City", "Execution time", "URL", id, "agencyId", "propertyDetailsUrl", "agencyName", "agencyProfilePageUrl", agent, bathrooms, bedrooms, carspaces, postcode, "propertyType", result, "streetName", "streetNumber", "streetType", suburb, "geoLocation.latitude", "geoLocation.longitude", "unitNumber", state, price) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s,